In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
def read_excel(path):
    return pd.read_excel(path)

def delete_outlier(df, HARI, JAM):
    df = df[(df['HARI'] == HARI) & (df['JAM'] == JAM)]    
    
    # Menghitung Q1 (kuartil pertama) dan Q3 (kuartil ketiga)
    Q1 = df['SMP'].quantile(0.25)
    Q3 = df['SMP'].quantile(0.75)
    IQR = Q3 - Q1
    
    # Menentukan batas bawah dan batas atas untuk outlier
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Menemukan dan menghapus outlier
    df_outlier = df[(df['SMP'] < lower_bound) | (df['SMP'] > upper_bound)]
    df_cleaned = df[(df['SMP'] >= lower_bound) & (df['SMP'] <= upper_bound)]
    
    # if(len(df_outlier)>0):
    #     print('Ada Outlier')

    # df_grouped = df_cleaned[['HARI', 'JAM', 'SMP']]
    # df_grouped = round(df_grouped.groupby(['HARI', 'JAM']).mean()).reset_index()

    return df_outlier, df_cleaned


In [3]:
path_1 = '../Data/Preprocessing/Data Losarang/2024/2_input_losarang_2024.xlsx'

df = read_excel(path_1)
df.head()

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2024-01-01 00:00:00,2024,1,1,Monday,00:00,78,30,12,120.0,62.4,30,30.0,122
1,2024-01-01 00:15:00,2024,1,1,Monday,00:15,97,32,9,138.0,77.6,32,22.5,132
2,2024-01-01 00:30:00,2024,1,1,Monday,00:30,98,41,7,146.0,78.4,41,17.5,137
3,2024-01-01 00:45:00,2024,1,1,Monday,00:45,88,31,12,131.0,70.4,31,30.0,131
4,2024-01-01 01:00:00,2024,1,1,Monday,01:00,56,28,12,96.0,44.8,28,30.0,103


In [4]:
df_waktu = df[['HARI','JAM']]
df_waktu = df_waktu.groupby(['HARI', 'JAM']).mean().round().reset_index()
df_waktu.head()

,HARI,JAM
0,Friday,00:00
1,Friday,00:15
2,Friday,00:30
3,Friday,00:45
4,Friday,01:00


In [5]:
all_df_outlier = []
all_df_cleaned = []

for i, row in df_waktu.iterrows():
    df_outlier,  df_cleaned = delete_outlier(df, row['HARI'], row['JAM'])
    all_df_outlier.append(df_outlier)
    all_df_cleaned.append(df_cleaned)

# gabungkan data
df_outlier_combined = pd.concat(all_df_outlier)
df_cleaned_combined = pd.concat(all_df_cleaned)

In [6]:
df_outlier_combined.head()

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
9121,2024-04-05 00:15:00,2024,4,5,Friday,00:15,326,85,88,499.0,260.8,85,220.0,566
9793,2024-04-12 00:15:00,2024,4,12,Friday,00:15,114,53,11,178.0,91.2,53,27.5,172
9122,2024-04-05 00:30:00,2024,4,5,Friday,00:30,397,91,79,567.0,317.6,91,197.5,606
9794,2024-04-12 00:30:00,2024,4,12,Friday,00:30,108,59,4,171.0,86.4,59,10.0,155
9123,2024-04-05 00:45:00,2024,4,5,Friday,00:45,326,88,63,477.0,260.8,88,157.5,506


In [7]:
df_cleaned_combined.head()

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
384,2024-01-05,2024,1,5,Friday,00:00,30,65,112,207.0,24.0,65,280.0,369
1056,2024-01-12,2024,1,12,Friday,00:00,40,43,45,128.0,32.0,43,112.5,188
1728,2024-01-19,2024,1,19,Friday,00:00,35,41,45,121.0,28.0,41,112.5,182
2400,2024-01-26,2024,1,26,Friday,00:00,20,54,89,163.0,16.0,54,222.5,292
3072,2024-02-02,2024,2,2,Friday,00:00,23,44,60,127.0,18.4,44,150.0,212


In [8]:
df_outlier_combined = df_outlier_combined.sort_values(by='WAKTU').reset_index(drop=True)
df_cleaned_combined = df_cleaned_combined.sort_values(by='WAKTU').reset_index(drop=True)

In [9]:
len(df_cleaned_combined) + len(df_outlier_combined)

11616

In [10]:
df_cleaned_combined.to_excel('../Data/Preprocessing/Data Losarang/2024/3_clean_losarang_2024.xlsx', index=False)
df_outlier_combined.to_excel('../Data/Preprocessing/Data Losarang/2024/3_outliers_losarang_2024.xlsx', index=False)

In [11]:
df_cleaned_combined.head()

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2024-01-01 00:00:00,2024,1,1,Monday,00:00,78,30,12,120.0,62.4,30,30.0,122
1,2024-01-01 00:15:00,2024,1,1,Monday,00:15,97,32,9,138.0,77.6,32,22.5,132
2,2024-01-01 00:30:00,2024,1,1,Monday,00:30,98,41,7,146.0,78.4,41,17.5,137
3,2024-01-01 00:45:00,2024,1,1,Monday,00:45,88,31,12,131.0,70.4,31,30.0,131
4,2024-01-01 01:00:00,2024,1,1,Monday,01:00,56,28,12,96.0,44.8,28,30.0,103


In [12]:
df_smp = df_cleaned_combined[['WAKTU','TAHUN', 'BULAN', 'TANGGAL', 'HARI','JAM','MOTOR','MOBIL','TRUK/BUS',
                              'SMP_MTR','SMP_MBL','SMP_TRK','SMP']]

In [13]:
df_smp.head(8)

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2024-01-01 00:00:00,2024,1,1,Monday,00:00,78,30,12,62.4,30,30.0,122
1,2024-01-01 00:15:00,2024,1,1,Monday,00:15,97,32,9,77.6,32,22.5,132
2,2024-01-01 00:30:00,2024,1,1,Monday,00:30,98,41,7,78.4,41,17.5,137
3,2024-01-01 00:45:00,2024,1,1,Monday,00:45,88,31,12,70.4,31,30.0,131
4,2024-01-01 01:00:00,2024,1,1,Monday,01:00,56,28,12,44.8,28,30.0,103
5,2024-01-01 01:15:00,2024,1,1,Monday,01:15,61,34,17,48.8,34,42.5,125
6,2024-01-01 01:30:00,2024,1,1,Monday,01:30,60,38,30,48.0,38,75.0,161
7,2024-01-01 01:45:00,2024,1,1,Monday,01:45,57,45,15,45.6,45,37.5,128


In [14]:
# Round down the WAKTU column to the nearest hour
df_smp_jam = df_smp.copy()
df_smp_jam['JAM'] = df_smp['WAKTU'].dt.floor('H')

# Drop the original WAKTU column if it's not needed anymore
df_smp_jam = df_smp_jam.drop(columns=['WAKTU'])

# Format the 'JAM' column to show only hour and minute
df_smp_jam['JAM'] = df_smp_jam['JAM'].dt.strftime('%H:%M')

# Group by the rounded-down hour and sum the SMP column
df_smp_jam = df_smp_jam.groupby(['TAHUN', 'BULAN', 'TANGGAL', 'HARI', 'JAM'], as_index=False).mean().round()

In [15]:
df_smp_jam.head(8)

,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2024,1,1,Monday,00:00,90.0,34.0,10.0,72.0,34.0,25.0,130.0
1,2024,1,1,Monday,01:00,58.0,36.0,18.0,47.0,36.0,46.0,129.0
2,2024,1,1,Monday,02:00,54.0,30.0,12.0,43.0,30.0,29.0,102.0
3,2024,1,1,Monday,03:00,42.0,27.0,13.0,34.0,27.0,33.0,94.0
4,2024,1,1,Monday,04:00,45.0,21.0,9.0,36.0,21.0,22.0,80.0
5,2024,1,1,Monday,05:00,52.0,20.0,14.0,42.0,20.0,36.0,98.0
6,2024,1,1,Monday,08:00,217.0,81.0,30.0,174.0,81.0,74.0,329.0
7,2024,1,1,Monday,09:00,258.0,102.0,33.0,206.0,102.0,82.0,390.0


In [16]:
# List kolom yang akan dikalikan 4
columns_to_multiply = ['MOTOR', 'MOBIL', 'TRUK/BUS', 'SMP_MTR', 'SMP_MBL', 'SMP_TRK', 'SMP']

# Mengalikan nilai kolom dengan 4
df_smp_jam[columns_to_multiply] = df_smp_jam[columns_to_multiply] * 4

In [17]:
df_lhrt = df_smp_jam.copy()

In [18]:
df_lhrt = df_lhrt.drop(columns=['TAHUN','BULAN','TANGGAL'])

In [19]:
df_lhrt.head()

,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,Monday,00:00,360.0,136.0,40.0,288.0,136.0,100.0,520.0
1,Monday,01:00,232.0,144.0,72.0,188.0,144.0,184.0,516.0
2,Monday,02:00,216.0,120.0,48.0,172.0,120.0,116.0,408.0
3,Monday,03:00,168.0,108.0,52.0,136.0,108.0,132.0,376.0
4,Monday,04:00,180.0,84.0,36.0,144.0,84.0,88.0,320.0


In [20]:
df_lhrt  = df_lhrt.groupby(['HARI', 'JAM'], as_index=False).mean().round()
df_lhrt.head()

,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,Friday,00:00,232.0,232.0,379.0,186.0,232.0,947.0,1364.0
1,Friday,01:00,254.0,246.0,386.0,204.0,246.0,965.0,1414.0
2,Friday,02:00,162.0,221.0,338.0,129.0,221.0,844.0,1194.0
3,Friday,03:00,108.0,210.0,337.0,86.0,210.0,840.0,1138.0
4,Friday,04:00,106.0,191.0,310.0,85.0,191.0,774.0,1051.0


In [21]:
df_lhrt.to_excel('../Data/Preprocessing/Data Losarang/2024/3_lhrt_losarang_2024.xlsx', index=False)

In [22]:
len(df_lhrt)

168